In [1]:
# Creating PyTorch Tensors

import torch

tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2, 3],
                         [4, 5, 6]])
tensor3d = torch.tensor([[[1,2], [3,4]],
                        [[5, 6], [7,8]]])

In [2]:
# Tensor data types

print(tensor1d.dtype)

floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.int64
torch.float32
torch.float32


In [3]:
# Tensor operations
print(tensor1d)

print(tensor2d.shape) # [2, 3] means 2 rows x 3 columns

# Reshaping with .reshape or .view
print(tensor2d.reshape(3,2))
print(tensor2d.view(3,2))

# Transposing .T
print(tensor2d.T)

# Matmul
print(tensor2d.matmul(tensor2d.T))

tensor([1, 2, 3])
torch.Size([2, 3])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[14, 32],
        [32, 77]])


In [4]:
# A logistic regresion forward pass
import torch.nn.functional as F
from torch.autograd import grad                         # Automatic differentiation engine

y = torch.tensor([1.0])                                 # true label
x1 = torch.tensor([1.1])                                # input feature
w1 = torch.tensor([2.2], requires_grad=True)            # weight
b = torch.tensor([0.0], requires_grad=True)             # bias

z = x1 * w1 + b                                         # net input
a = torch.sigmoid(z)                                    # activation and output

loss = F.binary_cross_entropy(a,y)

# Gradients
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [5]:
# Calculating all gradients via .backward
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [6]:
# Implementing multilayer NN

class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),        # Linear layers need in_features, out_features, and bias(default=True)
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [7]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {num_params}")

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
Total number of trainable parameters: 2213


In [8]:
# Examining layers: 
# Note: requires_grad=True means these are learnable weights
# 1st row corresponds to all connections to the 1st perceptron in the output layer, 2nd row to the 2nd, etc.
print(model.layers[4].weight)
print(model.layers[4].bias)

Parameter containing:
tensor([[-0.1692, -0.0858,  0.2063, -0.1134, -0.1537,  0.0388,  0.1798, -0.0861,
         -0.0695, -0.0770, -0.1165,  0.0031, -0.0258,  0.0932, -0.1665, -0.0642,
         -0.2058, -0.2228,  0.0619, -0.0289],
        [-0.2011, -0.0067,  0.0681,  0.1624,  0.1344, -0.2128,  0.0601, -0.2104,
          0.1657,  0.1458, -0.2062,  0.0016,  0.0697,  0.0092, -0.2019,  0.2094,
         -0.1101,  0.0695,  0.1398, -0.2045],
        [-0.0663, -0.1034,  0.2191, -0.0906, -0.1899, -0.2023, -0.0230,  0.1957,
         -0.1776, -0.1351, -0.2014,  0.2151,  0.1206,  0.0938,  0.1765,  0.0676,
          0.1493, -0.0508,  0.0346, -0.1222]], requires_grad=True)
Parameter containing:
tensor([-0.0535,  0.0220, -0.1022], requires_grad=True)


In [9]:
# Using the model
torch.manual_seed(123)
X = torch.rand((1,50))
out = model(X)
print(out)

# Note: Addmm is the last-used function to compute a variable in the computational graph
# mm (matrix multiplication) + addition (Add)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [10]:
# When we use a model for inference, we dont need to keep track of gradients -> save memory + computation
with torch.no_grad():
    out = model(X)
print(out)

# NOTE: It is common practice to return the last layer outputs (logits) without passing them to a nonlinear activation function
# This is because PyTorch's commonly used loss functions combine the softmax/sigmoid operation with negative log-likelihood in a 
# single class for numerical efficiency and stability.

# If we want to compute class-membership probabilities for our predictions, we have to call softmax function explicitly:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])
tensor([[0.3113, 0.3934, 0.2952]])


In [11]:
# Efficient Data Loaders
# MAIN IDEA: Custom Dataset class -> Defines how individual data recrods are loaded, and create Dataset Objects (test/train)
#            Custom DataLoader class -> Handless dataset shuffling, batching, etc.
#            Each Dataset object is fed to a DataLoader

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0, 1])

In [12]:
# Create custom Dataset class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        """ 
        Instructions for retrieving exactly one 
        data record and corresponding label 
        """
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        """
        Instructiosn for returning the total
        length of the dataset
        """
        return self.labels.shape[0]
    
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [13]:
# Create DataLoader class
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,           # ToyDataset instance is input to loader
    batch_size=2,               
    shuffle=True,               
    num_workers=0               # Number of background processes - 0: serial, >0: parallel -Use num_workers=4
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,              # Not necessary to shuffle a test dataset
    num_workers=0
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

# train_loader iterates over the training datset, visiting each training example exactly once -> training epoch

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [14]:
# In practice having a substantially smaller batch as the last batch in a trining epoch can disturb convergence during training.
# To prevent this, set drop_last=True to drop the last batch in each epoch

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,               
    shuffle=True,               
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)


Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [15]:
# Training Loop:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)      # The dataset has two features and two classes
optimizer = torch.optim.SGD(
    model.parameters(),                                 # The optimizer needs to know which parameters to optimize
    lr=0.5
)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)
        
        optimizer.zero_grad()                # Sets gradients from prev. round to 0 to prevent accumulation
        loss.backward()                      # Computes loss gradients given model parameters  
        optimizer.step()                     # Use gradients to update the model parameters

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train loss: {loss:.2f}"
        )   

model.eval()
# Insert optional model evaluation code

Epoch: 001/003 | Batch 000/002 | Train loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train loss: 0.00


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [16]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of learnable params: {num_params}")

Number of learnable params: 752


In [17]:
# Using the model to make predictions
model.eval()

# Get logits:
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

# Apply softmax
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

# Convert to class label predictions
predictions = torch.argmax(probas, dim=1)
print(predictions)

# Not neccesary to compute softmax probabilities to obtain class labels in this case:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])
tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])
tensor([0, 0, 0, 1, 1])


In [18]:
# Evaluation
print(predictions == y_train)

print(torch.sum(predictions==y_train))

tensor([True, True, True, True, True])
tensor(5)


In [19]:
# Accuracy
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions                 # Returns tensor of True/False
        correct += torch.sum(compare)                   # Counts True values
        total_examples += len(compare)

    return (correct / total_examples).item()            # .item() returns the value of the tensor as Python float

print(compute_accuracy(model, train_loader))

1.0


In [20]:
# Saving and loading models
torch.save(model.state_dict(), "model.pth")

# state_dict - Python dict that mapas each layer to trainable parameters
# "model.pth" - arbitrary filename

model = NeuralNetwork(2,2)                          # Instantiate the model before loading parameters
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [21]:
# Optimizing training performance with GPUs
print(torch.cuda.is_available())

# CPU 
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

# GPU using .to
tensor_1 = torch.tensor([1., 2., 3.]).to("cuda")        # If we had multiple GPUs, we can specify "cuda:0", etc.
tensor_2 = torch.tensor([4., 5., 6.]).to("cuda")
print(tensor_1 + tensor_2)

# NOTE: All tensors must be on the same device

True
tensor([5., 7., 9.])
tensor([5., 7., 9.], device='cuda:0')


In [22]:
# Single GPU Training:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)      

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train loss: {loss:.2f}"
        )   

model.eval()
# Insert optional model evaluation code

Epoch: 001/003 | Batch 000/002 | Train loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train loss: 0.00


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [23]:
# Time difference between CPU and GPU
dim = 100

a = torch.rand(dim, dim)
b = a.T
%timeit a @ b

a = a.to("cuda")
b = b.to("cuda")
%timeit a @ b


14.3 μs ± 91.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
9.58 μs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
